# Part 1

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}
base_url = 'https://www.cypherhunter.com/en/c/startups/'

The following cell is limited to scrap data from the companies on the provided webpage since scrapping each page sequentially is quite time consuming due to a large number of requests passed to the website.

In [3]:
company_names = []
category = []
description = []
likes_count = []
website = []

response = requests.get(base_url, headers=headers).text
         
soup = BeautifulSoup(response, 'lxml')
company_elements = soup.find_all('div', class_='ml-4')
            
for company in company_elements:
    name = company.div.text
    company_names.append(name)
    site_company_name = re.sub(r'[A-Z\s]', lambda x: x.group().lower() if x.group().isalpha() else '-', name)
    company_url = 'https://www.cypherhunter.com/en/p/' + site_company_name + '/'
    resp = requests.get(company_url, headers=headers).text
    sou = BeautifulSoup(resp, 'lxml')
    cat = sou.find_all('div',class_='flex flex-wrap gap-4')
    text = [element.strip() for element in cat[1].find_all(text=True)] if cat else []
    if len(text) == 0:
        category.append("NA")
    else:
        category.append(text)
    desc = sou.find('div',class_='mt-2 text-gray-500')
    description.append(desc.text if desc else "NA")
    likes = sou.find('span', class_='ml-2 inline-block text-sm font-medium')
    likes_count.append(likes.text if likes else "NA")
    web = sou.find('span', class_='inline max-phone:hidden lg:hidden')
    website.append(web.text if web else "NA")

C:\Users\manoj\AppData\Local\Temp\ipykernel_3648\392006136.py:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = [element.strip() for element in cat[1].find_all(text=True)] if cat else []


In [4]:
data = {
    'Company_Name':company_names,
    'Category':category,
    'Description': description,
    'Likes':likes_count,
    'Website':website
}

In [5]:
import pandas as pd
df = pd.DataFrame(data)
df

,Company_Name,Category,Description,Likes,Website
0,Moonwell,"[DeFi, Lending]",Lending made simple.,69,moonwell.fi
1,Ronin,"[DPoS, Gaming, Layer 1, Sidechain, zkVM]",Ethereum-linked sidechain made specifically fo...,7,roninchain.com
2,Sonic Labs,NA,NA,NA,NA
3,Lido,"[DeFi, PoS, Staking]",Develop and maintain decentralized liquid stak...,42,lido.fi
4,ENS,"[Coinbase, ENS, Naming Services, Non-Fungible ...",Ethereum Name Service.,14,ens.domains
5,The Open Network,"[Layer 1, Social]",The next gen network to unite all blockchains ...,8,ton.org
6,Surge,"[Swaps, Stablecoins]",Next-gen Decentralized Exchange on NEAR.,NA,surgefinance.xyz
7,Flappy Bird,[Gaming],Flappy Bird is back and bigger than ever. Clas...,1,flappybird.org
8,Trendies,NA,NA,NA,NA
9,Shuffle.com,NA,NA,NA,NA


The companies that do not have a website provided in the site is not removed from the dataframe because it's possible to identify potential leads from the company names also.

In [6]:
df.to_csv("company_data_part1.csv", index=False)